In [18]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
import os
from dotenv import load_dotenv
import yaml

load_dotenv(".env")

with open("agents/k8s_expert_agent/k8s_agent.yaml", "r") as file:
    settings = yaml.safe_load(file)

k8s_expert_agent_id = settings["agent_id"]

with open("agents/save_task_agent/k8s_agent.yaml", "r") as file:
    settings = yaml.safe_load(file)
save_k8s_task_agent_id = settings["agent_id"]

In [19]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT"),
    model=os.getenv("MODE_NAME"),
    api_version=os.getenv("API_VERSION"),
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    api_key=os.getenv("API_KEY"),
)

In [20]:

# from autogen_core.models import UserMessage
# result = await az_model_client.create(
#     [UserMessage(content="What is the capital of France?", source="user")]
# )
# print(result)

In [21]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [22]:
async def generate_k8s_task_agent(content: str) -> str:
    """
    Generate a task for the k8s expert agent
    """
    agent = project_client.agents.get_agent(k8s_expert_agent_id)

    thread = project_client.agents.create_thread()

    project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=content,
    )
    # create and execute a run
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
            # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # print the messages from the agent
    messages = project_client.agents.list_messages(thread_id=thread.id)
    # print(f"Messages: {messages}")

        # get the most recent message from the assistant
    last_msg =  messages.get_last_text_message_by_role("assistant")
    # if last_msg:
    #     print(f"Last Message: {last_msg.text.value}")
    return last_msg

In [23]:
async def rewrite_k8s_task_agent(content: str) -> str:
    """
    Validate the k8s task
    """
    prompt = f"""
    You are a kubernetes task rewriter, You need to rewrite the kubernetes task from web_surfer.
        1. It should be a valid kubernetes task.
        2. It must contain answer with kubectl command.
        3. It must contain yaml answer file.

        <Content>

         {content}

        </Content>

        Rewrite the task.
    """
    return await generate_k8s_task_agent(prompt)

In [24]:
write_agent = AssistantAgent(
    name="write_agent",
    model_client=az_model_client,
    tools=[rewrite_k8s_task_agent],
    system_message="""
        You are a kubernetes task writer, please help me write a kubernetes task based on bing search content."
    """
)

In [25]:
async def validator_k8s_task_agent(content: str) -> str:
    """
    Validate the k8s task
    """
    prompt = f"""
    You are a kubernetes task validator, You need to validate the kubernetes task written by the writer agent.
        1. It should be a valid kubernetes task.
        2. It must contain answer with kubectl command.
        3. It must contain yaml answer file.

        <Content>

         {content}

        </Content>

        Respond only 'Valid Kubernetes Task' or 'Invalid Kubernetes Task'.
    """
    return await generate_k8s_task_agent(prompt)

In [26]:
validator_agent = AssistantAgent(
    name="validator_agent",
    model_client=az_model_client,
    tools=[validator_k8s_task_agent],
    system_message="""
        You are a kubernetes task validator, You need to validate the kubernetes task written by the writer agent.
        1. It should be a valid kubernetes task.
        2. It must contain answer with kubectl command.
        3. It must contain yaml answer file.

        Respond only 'Valid Kubernetes Task' or 'Invalid Kubernetes Task'.
    """
)

In [27]:
async def save_task_agent(content: str) -> str:
    """
     Save the task content to a file and return the download link
    """   

    agent = project_client.agents.get_agent(save_k8s_task_agent_id)

    thread = project_client.agents.create_thread()

    project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="""
        
                    You are my Python programming assistant. Generate code,save """
        + content
        + """    
                    and execute it according to the following requirements

                    1. Save task content to task-{YYMMDDHHMMSS}.md

                    2. give me the download this file link
                """,
    )
    # create and execute a run
    run = project_client.agents.create_and_process_run(
        thread_id=thread.id, assistant_id=agent.id
    )
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # print the messages from the agent
    messages = project_client.agents.list_messages(thread_id=thread.id)
    # print(f"Messages: {messages}")

    # get the most recent message from the assistant
    last_msg = messages.get_last_text_message_by_role("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

        # print(f"File: {messages.file_path_annotations}")

    for file_path_annotation in messages.file_path_annotations:

        file_name = os.path.basename(file_path_annotation.text)

        project_client.agents.save_file(
            file_id=file_path_annotation.file_path.file_id,
            file_name=file_name,
            target_dir="./task",
        )

        # project_client.close()

    return "Saved"

In [28]:
save_k8s_task_agent = AssistantAgent(
    name="save_k8s_task_agent",
    model_client=az_model_client,
    tools=[save_task_agent],
    system_message="""
        Save only Valid Kubernetes Task from validator_agent. Respond with 'Saved' to when your task are saved.
    """
)

In [29]:
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

bing_search_agent = MultimodalWebSurfer(
        name="web_surfer",
        model_client=az_model_client
    )

In [30]:
text_termination = TextMentionTermination("Saved")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(10)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

In [31]:
reflection_team = RoundRobinGroupChat(
    [bing_search_agent, write_agent, validator_agent, save_k8s_task_agent],
    termination_condition=termination,
)

In [32]:
import warnings
warnings.filterwarnings("ignore")

In [33]:
await Console(
    reflection_team.run_stream(
        task="""
I am instructor to create kubernetes exercise task.
Help me write question and answer for the exercise tasks.
Kubernetes Entity must be self-contained in a namespace.

For example:
    Create a pod named 'nginx' with image 'nginx:latest' and expose it on port 80.
    
Answer:
    First use kubectl:

    kubectl create namespace my-namespace                               
    kubectl run nginx --image=nginx:latest --port=80 --namespace=my-namespace
                               
    Then use YAML file:

    apiVersion: v1
    kind: Namespace
    metadata:
        name: my-namespace
    ---              
    apiVersion: v1
    kind: Pod
    metadata:
        name: nginx
        namespace: my-namespace
    spec:                               
        containers:
        - name: nginx
          image: nginx:latest
          ports:
          - containerPort: 80
    
    
 Write the new question and answer.                            

    """
    )
)  # Stream the messages to the console.

---------- user ----------

I am instructor to create kubernetes exercise task.
Help me write question and answer for the exercise tasks.
Kubernetes Entity must be self-contained in a namespace.

For example:
    Create a pod named 'nginx' with image 'nginx:latest' and expose it on port 80.
    
Answer:
    First use kubectl:

    kubectl create namespace my-namespace                               
    kubectl run nginx --image=nginx:latest --port=80 --namespace=my-namespace
                               
    Then use YAML file:

    apiVersion: v1
    kind: Namespace
    metadata:
        name: my-namespace
    ---              
    apiVersion: v1
    kind: Pod
    metadata:
        name: nginx
        namespace: my-namespace
    spec:                               
        containers:
        - name: nginx
          image: nginx:latest
          ports:
          - containerPort: 80
    
    
 Write the new question and answer.                            

    
---------- web_surfer 

TaskResult(messages=[TextMessage(source='user', models_usage=None, content="\nI am instructor to create kubernetes exercise task.\nHelp me write question and answer for the exercise tasks.\nKubernetes Entity must be self-contained in a namespace.\n\nFor example:\n    Create a pod named 'nginx' with image 'nginx:latest' and expose it on port 80.\n    \nAnswer:\n    First use kubectl:\n\n    kubectl create namespace my-namespace                               \n    kubectl run nginx --image=nginx:latest --port=80 --namespace=my-namespace\n                               \n    Then use YAML file:\n\n    apiVersion: v1\n    kind: Namespace\n    metadata:\n        name: my-namespace\n    ---              \n    apiVersion: v1\n    kind: Pod\n    metadata:\n        name: nginx\n        namespace: my-namespace\n    spec:                               \n        containers:\n        - name: nginx\n          image: nginx:latest\n          ports:\n          - containerPort: 80\n    \n    \n Write th